## The Battle of Neighborhoods

Installing Libraries

In [1]:
#Install Beautifulsoup4
!pip install beautifulsoup4
#Install lxml
!pip install lxml
#Install Geopy
!pip install geopy
#Install Folium
!pip install folium
#Install Geocoder
!pip install geocoder

     |████████████████████████████████| 122kB 7.3MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 5.6MB/s eta 0:00:01
     |████████████████████████████████| 112kB 6.7MB/s eta 0:00:01
     |████████████████████████████████| 102kB 15.5MB/s ta 0:00:01


________________________________________

Import libraries

In [2]:
import folium
import requests 
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
import geopy
import geocoder

from bs4 import BeautifulSoup
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("Libraries imported!")

Libraries imported!


_____________________________________

API Foursquare

In [3]:
# @hidden_cell
# Foursquare API
CLIENT_ID = 'AZN3HATH0V2KAT5G0FQATZL1RROZYTH512MAIG2CKP0WPHLW' # Put Your Client Id
CLIENT_SECRET = 'JNA3DITD5YJYGFJFPIC4LCTMMG1145W4Y1QVGRYWWLA3HUBE' # Put You Client Secret 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: Hidden')
print('CLIENT_SECRET: Hidden')

Your credentails:
CLIENT_ID: Hidden
CLIENT_SECRET: Hidden


_________________________________

Scrapping data from "https://www.worldpostalcodes.org"

___________________________________________________________________________

Toluca City

In [4]:
List_url = "https://www.worldpostalcodes.org/en/mexico/municipality/list-of-postal-codes-in-toluca"
source = requests.get(List_url).text

Using BeautifulSoup

In [5]:
soup = BeautifulSoup(source, features="html")
table=soup.find('table')

Creating table

In [6]:
#Build dataframe with three columns: Postal Code, Postal Code Place Name, State and Municipality
column_names = ['Postal Code','Postal Code Place Name','State','Municipality']
df = pd.DataFrame(columns = column_names)
# Search all the Postal Code, Postal Code Place Name, State and Municipality 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==4:
        df.loc[len(df)] = row_data
#View DataFrame
df.head()

,Postal Code,Postal Code Place Name,State,Municipality
0,50000,Centro,México,Toluca
1,50010,La Magdalena,México,Toluca
2,50016,Parque industrial San Antonio,México,Toluca
3,50017,Las Brisas,México,Toluca
4,50018,El Mogote,México,Toluca


____________________________________________________________

Metepec City

In [7]:
List_url2= "https://www.worldpostalcodes.org/en/mexico/municipality/list-of-postal-codes-in-metepec-mexico"
source2= requests.get(List_url2).text

Using BeautifulSoup

In [8]:
soup2 = BeautifulSoup(source2, features="html")
table2=soup2.find('table')

Creating table

In [9]:
#Build dataframe with three columns: Postal Code, Postal Code Place Name, State and Municipality
column_names2= ['Postal Code','Postal Code Place Name','State','Municipality']
df2= pd.DataFrame(columns = column_names2)
# Search all the Postal Code, Postal Code Place Name, State and Municipality
for tr_cell_2 in table2.find_all('tr'):
    row_data2=[]
    for td_cell_2 in tr_cell_2.find_all('td'):
        row_data2.append(td_cell_2.text.strip())
    if len(row_data2)==4:
        df2.loc[len(df2)] = row_data2
#View DataFrame
df2.head()

,Postal Code,Postal Code Place Name,State,Municipality
0,52106,Francisco I. Madero,México,Metepec (México)
1,52140,Santiaguito,México,Metepec (México)
2,52143,San Miguel Totocuitlapilco,México,Metepec (México)
3,52144,Residencial Foresta,México,Metepec (México)
4,52145,San Lucas Tunco,México,Metepec (México)


___________________________________________________

Cleaning data

In [10]:
# Note: Both dataframes haven´t null spaces, I ommited this part
# Drop the column "State" in DF
df.drop('State', inplace=True, axis=1)
df.head()

,Postal Code,Postal Code Place Name,Municipality
0,50000,Centro,Toluca
1,50010,La Magdalena,Toluca
2,50016,Parque industrial San Antonio,Toluca
3,50017,Las Brisas,Toluca
4,50018,El Mogote,Toluca


In [11]:
# Drop the column "State" in DF2
df2.drop('State', inplace=True, axis=1)
df2.head()

,Postal Code,Postal Code Place Name,Municipality
0,52106,Francisco I. Madero,Metepec (México)
1,52140,Santiaguito,Metepec (México)
2,52143,San Miguel Totocuitlapilco,Metepec (México)
3,52144,Residencial Foresta,Metepec (México)
4,52145,San Lucas Tunco,Metepec (México)


In [12]:
# Replace the content of the column "Municipality" from "Metepec (México)" only to "Metepec" in DF2
df2['Municipality'] = df2['Municipality'].replace(['Metepec (México)'],'Metepec')
df2.head()

,Postal Code,Postal Code Place Name,Municipality
0,52106,Francisco I. Madero,Metepec
1,52140,Santiaguito,Metepec
2,52143,San Miguel Totocuitlapilco,Metepec
3,52144,Residencial Foresta,Metepec
4,52145,San Lucas Tunco,Metepec


In [13]:
# Replace the title of the columns "Postal Code" to "Postalcode", "Postal Code Place Name" to "Neighborhood" 
# and "Municipality" to "Borough" in both DF
df1_1= df.rename(columns={"Postal Code": "Postalcode", "Postal Code Place Name": "Neighborhood", "Municipality": "Borough"})
df2_1= df2.rename(columns={"Postal Code": "Postalcode", "Postal Code Place Name": "Neighborhood", "Municipality": "Borough"})

In [14]:
df1_1.head()

,Postalcode,Neighborhood,Borough
0,50000,Centro,Toluca
1,50010,La Magdalena,Toluca
2,50016,Parque industrial San Antonio,Toluca
3,50017,Las Brisas,Toluca
4,50018,El Mogote,Toluca


In [15]:
df2_1.head()

,Postalcode,Neighborhood,Borough
0,52106,Francisco I. Madero,Metepec
1,52140,Santiaguito,Metepec
2,52143,San Miguel Totocuitlapilco,Metepec
3,52144,Residencial Foresta,Metepec
4,52145,San Lucas Tunco,Metepec


In [16]:
# Check the shape of DF and DF2
print(df1_1.shape)
print(df2_1.shape)

(76, 3)
(31, 3)


_________________________________

Build tables

In [17]:
# Add coordenates to DF1

# Create a list to store neighbourhood data
neighborhoodList1 = df1_1['Neighborhood']
# Create a new DataFrame from the list
kl_df1 = pd.DataFrame({"Neighborhood": neighborhoodList1})
kl_df1.head()

,Neighborhood
0,Centro
1,La Magdalena
2,Parque industrial San Antonio
3,Las Brisas
4,El Mogote


In [18]:
# Defining a function to get coordinates
def get_latlng1(neighborhood):
    # initialize your variable to None
    lat_lng_coords1 = None
    # loop until you get the coordinates
    while(lat_lng_coords1 is None):
        g1 = geocoder.arcgis('{}, Toluca, Mexico'.format(neighborhood))
        lat_lng_coords1 = g1.latlng
    return lat_lng_coords1
# Call the function to get the coordinates, store in a new list using list comprehension
coords1 = [ get_latlng1(neighborhood) for neighborhood in kl_df1["Neighborhood"].tolist()]

In [19]:
# Create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords1 = pd.DataFrame(coords1, columns=['Latitude', 'Longitude'])
# Merge the coordinates into the DataFrame Neighborhood
kl_df1['Latitude'] = df_coords1['Latitude']
kl_df1['Longitude'] = df_coords1['Longitude']
kl_df1.head()

,Neighborhood,Latitude,Longitude
0,Centro,19.28916,-99.65575
1,La Magdalena,19.31871,-99.62997
2,Parque industrial San Antonio,19.30553,-99.61257
3,Las Brisas,19.30502,-99.61695
4,El Mogote,21.27222,-100.61944


In [20]:
# Merge the coordinates into the master DataFrame
df1_1['Latitude'] = df_coords1['Latitude']
df1_1['Longitude'] = df_coords1['Longitude']
print(df1_1.shape)
df1_1.head()

(76, 5)


,Postalcode,Neighborhood,Borough,Latitude,Longitude
0,50000,Centro,Toluca,19.28916,-99.65575
1,50010,La Magdalena,Toluca,19.31871,-99.62997
2,50016,Parque industrial San Antonio,Toluca,19.30553,-99.61257
3,50017,Las Brisas,Toluca,19.30502,-99.61695
4,50018,El Mogote,Toluca,21.27222,-100.61944


In [21]:
# Get the coordinates of Toluca, Mexico
address = 'Toluca, Mexico'
geolocator = Nominatim(user_agent="smy-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toluca, Mexico are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toluca, Mexico are 19.292545, -99.6569007.


In [22]:
#Map of Toluca
map_toluca = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(df1_1['Latitude'], df1_1['Longitude'], df1_1['Borough'], df1_1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toluca)
map_toluca

In [23]:
# Add coordenates to DF2

# Create a list to store neighbourhood data
neighborhoodList2 = df2_1['Neighborhood']
# Create a new DataFrame from the list
kl_df2 = pd.DataFrame({"Neighborhood": neighborhoodList2})
kl_df2.head()

,Neighborhood
0,Francisco I. Madero
1,Santiaguito
2,San Miguel Totocuitlapilco
3,Residencial Foresta
4,San Lucas Tunco


In [24]:
# Defining a function to get coordinates
def get_latlng2(neighborhood):
    # initialize your variable to None
    lat_lng_coords2 = None
    # loop until you get the coordinates
    while(lat_lng_coords2 is None):
        g2 = geocoder.arcgis('{}, Metepec, Mexico'.format(neighborhood))
        lat_lng_coords2 = g2.latlng
    return lat_lng_coords2
# Call the function to get the coordinates, store in a new list using list comprehension
coords2 = [ get_latlng2(neighborhood) for neighborhood in kl_df2["Neighborhood"].tolist()]

In [25]:
# Create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords2 = pd.DataFrame(coords2, columns=['Latitude', 'Longitude'])
# Merge the coordinates into the DataFrame Neighborhood
kl_df2['Latitude'] = df_coords2['Latitude']
kl_df2['Longitude'] = df_coords2['Longitude']
kl_df2.head()

,Neighborhood,Latitude,Longitude
0,Francisco I. Madero,19.28477,-99.56766
1,Santiaguito,19.24891,-99.58096
2,San Miguel Totocuitlapilco,19.23598,-99.60242
3,Residencial Foresta,25.82912,-100.11278
4,San Lucas Tunco,19.23132,-99.53478


In [26]:
# Merge the coordinates into the master DataFrame
df2_1['Latitude'] = df_coords2['Latitude']
df2_1['Longitude'] = df_coords2['Longitude']
print(df2_1.shape)
df2_1.head()

(31, 5)


,Postalcode,Neighborhood,Borough,Latitude,Longitude
0,52106,Francisco I. Madero,Metepec,19.28477,-99.56766
1,52140,Santiaguito,Metepec,19.24891,-99.58096
2,52143,San Miguel Totocuitlapilco,Metepec,19.23598,-99.60242
3,52144,Residencial Foresta,Metepec,25.82912,-100.11278
4,52145,San Lucas Tunco,Metepec,19.23132,-99.53478


In [27]:
# Get the coordinates of Metepec, Mexico
address = 'Metepec, Mexico'
geolocator = Nominatim(user_agent="smy-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Metepec, Mexico are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Metepec, Mexico are 19.25113375, -99.5937281303492.


In [28]:
#Map of Metepec
map_metepec = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(df2_1['Latitude'], df2_1['Longitude'], df2_1['Borough'], df2_1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_metepec)
map_metepec

# Exploring and clustering the neighborhoods in Toluca.

For this project, I work with neighborhoods near to Downtown, I choose this neighborhoods manually because not have clear indicators or keywords for group automatically.

In [29]:
# Search index of Neighborhood in Toluca downtown
df1_1[df1_1['Neighborhood'].str.contains("Centro")]
df1_1[df1_1['Neighborhood'].str.contains("Santa Clara")]
df1_1[df1_1['Neighborhood'].str.contains("El Cóporo")]
df1_1[df1_1['Neighborhood'].str.contains("La Retama")]
df1_1[df1_1['Neighborhood'].str.contains("Alcaltunco")]
df1_1[df1_1['Neighborhood'].str.contains("San Bernardino")]
df1_1[df1_1['Neighborhood'].str.contains("Morelos 1a Sección")]
df1_1[df1_1['Neighborhood'].str.contains("Américas")]
df1_1[df1_1['Neighborhood'].str.contains("Paseo Tollocan INFONAVIT")]

,Postalcode,Neighborhood,Borough,Latitude,Longitude
24,50123,Paseo Tollocan INFONAVIT,Toluca,19.27294,-99.659891


In [30]:
# Get the boroughs near to Toluca downtown.
rows1=[0,17,10,9,72,16,22,25,24]
cols1=["Neighborhood","Postalcode","Latitude","Longitude"]
toluca_data=df1_1.loc[rows1,cols1]
toluca_data

,Neighborhood,Postalcode,Latitude,Longitude
0,Centro,50000,19.28916,-99.655750
17,Santa Clara,50090,19.29187,-99.649360
10,El Cóporo,50050,19.29568,-99.659140
9,La Retama,50040,19.29394,-99.661080
72,Alcaltunco,50290,19.28567,-99.652810
16,San Bernardino,50080,19.28901,-99.668520
22,Morelos 1a Sección,50120,19.28338,-99.668880
25,Américas,50130,19.28196,-99.645300
24,Paseo Tollocan INFONAVIT,50123,19.27294,-99.659891


Map of Neighborhoods near to Toluca Downtown

In [31]:
map_toluca1 = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(toluca_data['Latitude'], toluca_data['Longitude'], toluca_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7,parse_html=False).add_to(map_toluca1)  
map_toluca1

# Exploring and clustering the neighborhoods in Metepec.

For this project, I work with neighborhoods near to Downtown, I choose this neighborhoods manually because not have clear indicators or keywords for group automatically.

In [32]:
# Search index of Neighborhood in Metepec downtown
df2_1[df2_1['Neighborhood'].str.contains("Fortanet")]
df2_1[df2_1['Neighborhood'].str.contains("Las Jaras")]
df2_1[df2_1['Neighborhood'].str.contains("Luisa Isabel Campos de Jiménez Cantú")]
df2_1[df2_1['Neighborhood'].str.contains("Villas Alteza")]
df2_1[df2_1['Neighborhood'].str.contains("Jesús Jiménez Gallardo")]
df2_1[df2_1['Neighborhood'].str.contains("Purísima")]
df2_1[df2_1['Neighborhood'].str.contains("La Providencia")]
df2_1[df2_1['Neighborhood'].str.contains("Rincón de San Gabriel")]
df2_1[df2_1['Neighborhood'].str.contains("Virreyes Residencial")]
df2_1[df2_1['Neighborhood'].str.contains("Hípico")]
df2_1[df2_1['Neighborhood'].str.contains("El Rodeo")]

,Postalcode,Neighborhood,Borough,Latitude,Longitude
13,52157,El Rodeo,Metepec,19.26708,-99.63629


In [33]:
# Get the boroughs near to Toluca downtown.
rows2=[15,16,20,22,21,23,28,14,11,12,13]
cols2=["Neighborhood","Postalcode","Latitude","Longitude"]
metepec_data=df2_1.loc[rows2,cols2]
metepec_data

,Neighborhood,Postalcode,Latitude,Longitude
15,Fortanet,52159,19.26218,-99.61466
16,Las Jaras,52160,19.25438,-99.62942
20,Luisa Isabel Campos de Jiménez Cantú,52166,19.25204,-99.63385
22,Villas Alteza,52168,19.25733,-99.63929
21,Jesús Jiménez Gallardo,52167,19.25795,-99.63389
23,Purísima,52169,19.26197,-99.62808
28,La Providencia,52177,19.27093,-99.60437
14,Rincón de San Gabriel,52158,19.27258,-99.62310
11,Virreyes Residencial,52155,19.26888,-99.63082
12,Hípico,52156,19.26689,-99.63202


In [34]:
map_metepec1 = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(metepec_data['Latitude'], metepec_data['Longitude'], metepec_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7,parse_html=False).add_to(map_metepec1)  
map_metepec1

______________________________

1.- Exploring Neighbourhood in Toluca

In [35]:
# Exploring using Foursquare
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([( name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [36]:
df3=toluca_data
toluca_venues = getNearbyVenues(names=df3['Neighborhood'], latitudes=df3['Latitude'],longitudes=df3['Longitude'])

Centro
Santa Clara
El Cóporo
La Retama
Alcaltunco
San Bernardino
Morelos 1a Sección
Américas
Paseo Tollocan INFONAVIT


In [37]:
print(toluca_venues.shape)
toluca_venues.head()

(230, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Centro,19.28916,-99.65575,Bistró Mecha Centro Histórico,19.290751,-99.654963,Bistro
1,Centro,19.28916,-99.65575,Fantasias Miguel,19.290071,-99.655655,Shopping Mall
2,Centro,19.28916,-99.65575,The Green Factory,19.289671,-99.657675,Salad Place
3,Centro,19.28916,-99.65575,Centro Historico,19.289986,-99.655161,Plaza
4,Centro,19.28916,-99.65575,Café Catalán,19.289049,-99.656387,Café


In [38]:
toluca_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alcaltunco,28,28,28,28,28,28
Américas,22,22,22,22,22,22
Centro,30,30,30,30,30,30
El Cóporo,17,17,17,17,17,17
La Retama,28,28,28,28,28,28
Morelos 1a Sección,27,27,27,27,27,27
Paseo Tollocan INFONAVIT,18,18,18,18,18,18
San Bernardino,30,30,30,30,30,30
Santa Clara,30,30,30,30,30,30


In [39]:
# Count categories
print('There are {} uniques categories.'.format(len(toluca_venues['Venue Category'].unique())))

There are 72 uniques categories.


2.- Analyze Each Borough Neighborhood

In [40]:
toluca_onehot = pd.get_dummies(toluca_venues[['Venue Category']], prefix="", prefix_sep="")

toluca_onehot['Neighborhood'] = toluca_venues['Neighborhood'] 

fixed_columns1 = [toluca_onehot.columns[-1]] + list(toluca_onehot.columns[:-1])
toluca_onehot = toluca_onehot[fixed_columns1]
toluca_onehot.head()

,Neighborhood,Amphitheater,Argentinian Restaurant,Art Museum,Asian Restaurant,BBQ Joint,Bakery,Bar,Bistro,Bookstore,Breakfast Spot,Burger Joint,Café,Chocolate Shop,Church,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Creperie,Department Store,Dessert Shop,Diner,Dog Run,Drugstore,Electronics Store,Fast Food Restaurant,Food,Food Court,Food Truck,Fried Chicken Joint,Gym,Gym / Fitness Center,Gym Pool,Historic Site,History Museum,Hot Dog Joint,Hotel,Ice Cream Shop,Internet Cafe,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Liquor Store,Mexican Restaurant,Miscellaneous Shop,Newsstand,Nightclub,Optical Shop,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Plaza,Record Shop,Restaurant,Salad Place,Sandwich Place,Science Museum,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,Spa,Steakhouse,Sushi Restaurant,Taco Place,Tea Room,Theater,Vegetarian / Vegan Restaurant,Yoga Studio
0,Centro,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Centro,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
toluca_onehot.shape

(230, 73)

In [42]:
# Grouped by Neighborhood
toluca_grouped = toluca_onehot.groupby('Neighborhood').mean().reset_index()
toluca_grouped

,Neighborhood,Amphitheater,Argentinian Restaurant,Art Museum,Asian Restaurant,BBQ Joint,Bakery,Bar,Bistro,Bookstore,Breakfast Spot,Burger Joint,Café,Chocolate Shop,Church,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Creperie,Department Store,Dessert Shop,Diner,Dog Run,Drugstore,Electronics Store,Fast Food Restaurant,Food,Food Court,Food Truck,Fried Chicken Joint,Gym,Gym / Fitness Center,Gym Pool,Historic Site,History Museum,Hot Dog Joint,Hotel,Ice Cream Shop,Internet Cafe,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Liquor Store,Mexican Restaurant,Miscellaneous Shop,Newsstand,Nightclub,Optical Shop,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Plaza,Record Shop,Restaurant,Salad Place,Sandwich Place,Science Museum,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,Spa,Steakhouse,Sushi Restaurant,Taco Place,Tea Room,Theater,Vegetarian / Vegan Restaurant,Yoga Studio
0,Alcaltunco,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.035714,0.000000,0.035714,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.107143,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.035714,0.000000,0.000000,0.035714,0.071429,0.035714,0.035714,0.071429,0.000000,0.035714,0.000000,0.000000,0.000000,0.035714,0.000000,0.071429,0.000000,0.000000,0.035714,0.000000
1,Américas,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.409091,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.136364,0.000000,0.000000,0.000000,0.000000
2,Centro,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.066667,0.000000,0.033333,0.033333,0.033333,0.033333,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.066667,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.033333,0.066667,0.033333,0.066667,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000
3,El Cóporo,0.000000,0.000000,0.235294,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.058824,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000
4,La Retama,0.000000,0.000000,0.107143,0.000000,0.000000,0.035714,0.000000,0.000000,0.035714,0.035714,0.000000,0.035714,0.000000,0.035714,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.035714,0.000000,0.000000,0.035714,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.035714,0.142857,0.00000

In [43]:
toluca_grouped.shape

(9, 73)

In [44]:
# Top 5 grouped by Neighborhood
num_top_venues = 5
for neigh in toluca_grouped['Neighborhood']:
    print("----"+neigh+"----")
    temp1 = toluca_grouped[toluca_grouped['Neighborhood'] == neigh].T.reset_index()
    temp1.columns = ['venue','freq']
    temp1 = temp1.iloc[1:]
    temp1['freq'] = temp1['freq'].astype(float)
    temp1 = temp1.round({'freq': 2})
    print(temp1.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alcaltunco----
                venue  freq
0      Ice Cream Shop  0.11
1  Mexican Restaurant  0.07
2          Taco Place  0.07
3         Salad Place  0.07
4                Café  0.07


----Américas----
                           venue  freq
0             Mexican Restaurant  0.41
1                     Taco Place  0.14
2              Convenience Store  0.09
3  Paper / Office Supplies Store  0.09
4                    Salad Place  0.05


----Centro----
            venue  freq
0      Restaurant  0.07
1  Breakfast Spot  0.07
2  Ice Cream Shop  0.07
3  Sandwich Place  0.07
4           Plaza  0.07


----El Cóporo----
          venue  freq
0    Art Museum  0.24
1   Coffee Shop  0.12
2         Hotel  0.06
3          Café  0.06
4  Liquor Store  0.06


----La Retama----
                venue  freq
0  Mexican Restaurant  0.14
1          Art Museum  0.11
2         Coffee Shop  0.07
3          Taco Place  0.07
4               Hotel  0.04


----Morelos 1a Sección----
                venue  freq
0 

3.- Clustering Neighborhoods

In [45]:
# Preprocesing
# Defining most common venues
def return_most_common_venues1(row1, num_top_venues1):
    row_categories1 = row1.iloc[1:]
    row_categories_sorted1 = row_categories1.sort_values(ascending=False)
    return row_categories_sorted1.index.values[0:num_top_venues1]

In [46]:
# Creating arrange of neighborhoods sorted

import numpy as np
num_top_venues1 = 5
indicators1 = ['st', 'nd', 'rd']

columns1 = ['Neighborhood']
for ind1 in np.arange(num_top_venues):
    try:
        columns1.append('{}{} Most Common Venue'.format(ind1+1, indicators1[ind1]))
    except:
        columns1.append('{}th Most Common Venue'.format(ind1+1))

neighborhoods_venues_sorted1 = pd.DataFrame(columns=columns1)
neighborhoods_venues_sorted1['Neighborhood'] = toluca_grouped['Neighborhood']

for ind1 in np.arange(toluca_grouped.shape[0]):
    neighborhoods_venues_sorted1.iloc[ind1, 1:] = return_most_common_venues1(toluca_grouped.iloc[ind1,:], num_top_venues1)

neighborhoods_venues_sorted1.shape

(9, 6)

In [47]:
# Import Cluster k-means
import sklearn.cluster.k_means_
km1 = KMeans(n_clusters=3, init='k-means++', max_iter=100, n_init=1, 
  verbose=True)

In [48]:
# Clustering
kclusters1 = 5
toluca_grouped_clustering = toluca_grouped.drop('Neighborhood', 1)
kmeans1 = KMeans(n_clusters=kclusters1, random_state=1).fit(toluca_grouped_clustering)
print(kmeans1.labels_[0:5])
print(len(kmeans1.labels_))

[0 3 0 2 2]
9


In [49]:
df3.head()

,Neighborhood,Postalcode,Latitude,Longitude
0,Centro,50000,19.28916,-99.65575
17,Santa Clara,50090,19.29187,-99.64936
10,El Cóporo,50050,19.29568,-99.65914
9,La Retama,50040,19.29394,-99.66108
72,Alcaltunco,50290,19.28567,-99.65281


In [50]:
neighborhoods_venues_sorted1.insert(0, 'Cluster Labels', kmeans1.labels_)
toluca_merged = df3
toluca_merged = toluca_merged.join(neighborhoods_venues_sorted1.set_index('Neighborhood'), on='Neighborhood')

toluca_merged.head() # check the last columns!

,Neighborhood,Postalcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Centro,50000,19.28916,-99.65575,0,Restaurant,Ice Cream Shop,Breakfast Spot,Plaza,Sandwich Place
17,Santa Clara,50090,19.29187,-99.64936,1,Seafood Restaurant,Gym,Vegetarian / Vegan Restaurant,Taco Place,BBQ Joint
10,El Cóporo,50050,19.29568,-99.65914,2,Art Museum,Coffee Shop,Fast Food Restaurant,Theater,Historic Site
9,La Retama,50040,19.29394,-99.66108,2,Mexican Restaurant,Art Museum,Coffee Shop,Taco Place,Breakfast Spot
72,Alcaltunco,50290,19.28567,-99.65281,0,Ice Cream Shop,Salad Place,Mexican Restaurant,Taco Place,Café


4.- Visualization of the resulting clusters

In [51]:
map_clusterst = folium.Map(location=[latitude, longitude], zoom_start=11)

x1 = np.arange(kclusters1)
ys1 = [i+x1+(i*x1)**2 for i in range(kclusters1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys1)))
rainbow1 = [colors.rgb2hex(i) for i in colors_array]

markers_colors1 = []
for lat, lon, poi, cluster in zip(toluca_merged['Latitude'], toluca_merged['Longitude'], toluca_merged['Neighborhood'],kmeans1.labels_):
    label1 = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label1, color=rainbow1[cluster-1], fill=True, fill_color=rainbow1[cluster-1], fill_opacity=0.7).add_to(map_clusterst)
map_clusterst

1b.- Exploring Neighbourhood in Metepec

In [52]:
df4=metepec_data
metepec_venues = getNearbyVenues(names=df4['Neighborhood'], latitudes=df4['Latitude'],longitudes=df4['Longitude'])

Fortanet
Las Jaras
Luisa Isabel Campos de Jiménez Cantú
Villas Alteza
Jesús Jiménez Gallardo
Purísima
La Providencia
Rincón de San Gabriel
Virreyes Residencial
Hípico
El Rodeo


In [53]:
print(metepec_venues.shape)
metepec_venues.head()

(211, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Fortanet,19.26218,-99.61466,Cielito Querido Café,19.261164,-99.613279,Café
1,Fortanet,19.26218,-99.61466,Sports World,19.260901,-99.613814,Gym / Fitness Center
2,Fortanet,19.26218,-99.61466,Starbucks,19.261204,-99.617083,Coffee Shop
3,Fortanet,19.26218,-99.61466,Unidad Deportiva ( Hortaliza ),19.265195,-99.613503,Park
4,Fortanet,19.26218,-99.61466,Mimis,19.261639,-99.611945,Theme Restaurant


In [54]:
metepec_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
El Rodeo,30,30,30,30,30,30
Fortanet,18,18,18,18,18,18
Hípico,22,22,22,22,22,22
Jesús Jiménez Gallardo,21,21,21,21,21,21
La Providencia,29,29,29,29,29,29
Las Jaras,9,9,9,9,9,9
Luisa Isabel Campos de Jiménez Cantú,6,6,6,6,6,6
Purísima,30,30,30,30,30,30
Rincón de San Gabriel,11,11,11,11,11,11


In [55]:
# Count categories
print('There are {} uniques categories.'.format(len(metepec_venues['Venue Category'].unique())))

There are 65 uniques categories.


2.- Analyze Each Borough Neighborhood

In [56]:
metepec_onehot = pd.get_dummies(metepec_venues[['Venue Category']], prefix="", prefix_sep="")

metepec_onehot['Neighborhood'] = metepec_venues['Neighborhood'] 

fixed_columns2 = [metepec_onehot.columns[-1]] + list(metepec_onehot.columns[:-1])
metepec_onehot = metepec_onehot[fixed_columns2]
metepec_onehot.head()

,Neighborhood,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bar,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Burger Joint,Café,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dance Studio,Discount Store,Donut Shop,Electronics Store,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Furniture / Home Store,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Ice Cream Shop,Italian Restaurant,Liquor Store,Market,Mexican Restaurant,Movie Theater,Moving Target,New American Restaurant,Nightclub,Park,Pet Store,Pharmacy,Pizza Place,Pub,Restaurant,Salad Place,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tex-Mex Restaurant,Thai Restaurant,Theme Park,Theme Restaurant,Vegetarian / Vegan Restaurant
0,Fortanet,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Fortanet,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Fortanet,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Fortanet,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Fortanet,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [57]:
metepec_onehot.shape

(211, 66)

In [58]:
# Grouped by Neighborhood
metepec_grouped = metepec_onehot.groupby('Neighborhood').mean().reset_index()
metepec_grouped

,Neighborhood,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bar,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Burger Joint,Café,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dance Studio,Discount Store,Donut Shop,Electronics Store,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Furniture / Home Store,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Ice Cream Shop,Italian Restaurant,Liquor Store,Market,Mexican Restaurant,Movie Theater,Moving Target,New American Restaurant,Nightclub,Park,Pet Store,Pharmacy,Pizza Place,Pub,Restaurant,Salad Place,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tex-Mex Restaurant,Thai Restaurant,Theme Park,Theme Restaurant,Vegetarian / Vegan Restaurant
0,El Rodeo,0.000000,0.000000,0.033333,0.100000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.233333,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.066667,0.000000,0.000000,0.066667,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000
1,Fortanet,0.055556,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.055556,0.111111,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.055556,0.000000,0.000000,0.055556,0.055556,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000
2,Hípico,0.000000,0.000000,0.045455,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.272727,0.000000,0.000000,0.045455,0.000000,0.045455,0.000000,0.045455,0.000000,0.000000,0.045455,0.000000,0.136364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000
3,Jesús Jiménez Gallardo,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.095238,0.000000,0.000000,0.000000,0.238095,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.095238,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.238095,0.000000,0.000000,0.000000,0.000000,0.047619
4,La Providencia,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.034483,0.034483,0.000000,0.034483,0.034483,0.034483,0.103448,0.034483,0.034483,0.034483,0.034483,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.068966,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.034483,0.034483,0.000000,0.000000,0.034483,0.000000,0.000000,0.068966,0.000000,0.000000,0.034483,0.000000,0.034483,0.000000,0.000000
5,Las Jaras,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [59]:
metepec_grouped.shape

(11, 66)

In [60]:
# Top 5 grouped by Neighborhood
num_top_venues2 = 5
for neigh2 in metepec_grouped['Neighborhood']:
    print("----"+neigh2+"----")
    temp2 = metepec_grouped[metepec_grouped['Neighborhood'] == neigh2].T.reset_index()
    temp2.columns = ['venue','freq']
    temp2 = temp2.iloc[1:]
    temp2['freq'] = temp2['freq'].astype(float)
    temp2 = temp2.round({'freq': 2})
    print(temp2.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues2))
    print('\n')

----El Rodeo----
                venue  freq
0  Mexican Restaurant  0.23
1          Taco Place  0.13
2              Bakery  0.10
3            Pharmacy  0.07
4          Restaurant  0.07


----Fortanet----
                  venue  freq
0  Gym / Fitness Center  0.11
1            Food Court  0.06
2      Theme Restaurant  0.06
3           Supermarket  0.06
4          Soccer Field  0.06


----Hípico----
                venue  freq
0  Mexican Restaurant  0.27
1  Seafood Restaurant  0.14
2          Taco Place  0.09
3          Restaurant  0.05
4   Electronics Store  0.05


----Jesús Jiménez Gallardo----
                           venue  freq
0                     Taco Place  0.24
1             Mexican Restaurant  0.24
2             Seafood Restaurant  0.10
3                 Ice Cream Shop  0.10
4  Vegetarian / Vegan Restaurant  0.05


----La Providencia----
               venue  freq
0        Coffee Shop  0.10
1        Supermarket  0.07
2      Movie Theater  0.07
3  Convenience Store  0.03
4   

3.- Clustering Neighborhoods

In [61]:
# Preprocesing
# Defining most common venues
def return_most_common_venues2(row2, num_top_venues2):
    row_categories2 = row2.iloc[1:]
    row_categories_sorted2 = row_categories2.sort_values(ascending=False)
    return row_categories_sorted2.index.values[0:num_top_venues2]

In [62]:
# Creating arrange of neighborhoods sorted

import numpy as np
num_top_venues2 = 5
indicators2 = ['st', 'nd', 'rd']

columns2 = ['Neighborhood']
for ind2 in np.arange(num_top_venues):
    try:
        columns2.append('{}{} Most Common Venue'.format(ind2+1, indicators2[ind2]))
    except:
        columns2.append('{}th Most Common Venue'.format(ind2+1))

neighborhoods_venues_sorted2 = pd.DataFrame(columns=columns2)
neighborhoods_venues_sorted2['Neighborhood'] = metepec_grouped['Neighborhood']

for ind2 in np.arange(metepec_grouped.shape[0]):
    neighborhoods_venues_sorted2.iloc[ind2, 1:] = return_most_common_venues2(metepec_grouped.iloc[ind2,:], num_top_venues2)

neighborhoods_venues_sorted2.shape

(11, 6)

In [63]:
# Import Cluster k-means
import sklearn.cluster.k_means_
km2 = KMeans(n_clusters=3, init='k-means++', max_iter=100, n_init=1, 
  verbose=True)

In [64]:
# Clustering
kclusters2 = 5
metepec_grouped_clustering = metepec_grouped.drop('Neighborhood', 1)
kmeans2 = KMeans(n_clusters=kclusters2, random_state=1).fit(metepec_grouped_clustering)
print(kmeans2.labels_[0:5])
print(len(kmeans2.labels_))

[4 2 4 0 2]
11


In [65]:
df4.head()

,Neighborhood,Postalcode,Latitude,Longitude
15,Fortanet,52159,19.26218,-99.61466
16,Las Jaras,52160,19.25438,-99.62942
20,Luisa Isabel Campos de Jiménez Cantú,52166,19.25204,-99.63385
22,Villas Alteza,52168,19.25733,-99.63929
21,Jesús Jiménez Gallardo,52167,19.25795,-99.63389


In [66]:
neighborhoods_venues_sorted2.insert(0, 'Cluster Labels', kmeans2.labels_)
metepec_merged = df4
metepec_merged = metepec_merged.join(neighborhoods_venues_sorted2.set_index('Neighborhood'), on='Neighborhood')

metepec_merged.head() # check the last columns!

,Neighborhood,Postalcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
15,Fortanet,52159,19.26218,-99.61466,2,Gym / Fitness Center,Seafood Restaurant,BBQ Joint,Café,Coffee Shop
16,Las Jaras,52160,19.25438,-99.62942,1,Gym,Taco Place,Coffee Shop,Restaurant,Café
20,Luisa Isabel Campos de Jiménez Cantú,52166,19.25204,-99.63385,0,Taco Place,Convenience Store,Seafood Restaurant,Flea Market,Mexican Restaurant
22,Villas Alteza,52168,19.25733,-99.63929,0,Seafood Restaurant,Mexican Restaurant,Gym / Fitness Center,Burger Joint,Gym
21,Jesús Jiménez Gallardo,52167,19.25795,-99.63389,0,Taco Place,Mexican Restaurant,Ice Cream Shop,Seafood Restaurant,Vegetarian / Vegan Restaurant


4.- Visualization of the resulting clusters

In [67]:
map_clustersm = folium.Map(location=[latitude, longitude], zoom_start=11)

x2 = np.arange(kclusters2)
ys2 = [i+x2+(i*x1)**2 for i in range(kclusters2)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys2)))
rainbow2 = [colors.rgb2hex(i) for i in colors_array]

markers_colors2 = []
for lat, lon, poi, cluster in zip(metepec_merged['Latitude'], metepec_merged['Longitude'], metepec_merged['Neighborhood'],kmeans2.labels_):
    label2 = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label2, color=rainbow2[cluster-1], fill=True, fill_color=rainbow2[cluster-1], fill_opacity=0.7).add_to(map_clustersm)
map_clustersm